# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [1]:
# Imports
import numpy as np
import pandas as pd
from math import log
import nltk

In [3]:
class NaiveBayesClassificator:
    
    # Initializing the classificator, which can be customized
    def __init__(self, n_gram=1, stem=True, stop_words=True, alpha=1, class_prob=None):
        # Variable that defines the size of the gram to be used
        self.n_gram = n_gram
        # Boolean that defines if the words will be stemmized or not
        self.stem = stem
        # Boolean that defines if stop_words will be removed
        self.stop_words = stop_words
        # Variable that defines which alpha will be used for the Smoothing of probabilities.
        self.alpha = alpha
        """
        class-prob defines how the probability of a specific classification (P(class)) will be calculated
        if None -> P(category) = (number of appearances of the category)/(total)
        if "equal" -> P(category) = 1/(number of categories)
        """
        self.class_prob = class_prob
        # Stemmer that is used if words are to be stemmed
        self.stemmer = nltk.stem.RSLPStemmer()
        # List with the stop-words
        self.stop_words_list = nltk.corpus.stopwords.words('portuguese')
    
    # This function is utilized to clean the sentence
    def _clean_sentence(self, sentence):
        
        # Error occurred while testing. String created to avoid errors
        string = str(sentence)
    
        # Cleaning unwanted characters on the sentence
        string = string.replace(":", " ")
        string = string.replace(";", " ")
        string = string.replace(",", " ")
        string = string.replace("?", " ")
        string = string.replace("(", " ")
        string = string.replace(")", " ")
        string = string.replace("\n", " ")
        string = string.replace("'", " ")
        string = string.replace(".", " ")
        string = string.replace('"', " ")
        string = string.replace("!", " ")
        string = string.replace("@", " ")
        #l Lowercasing all letters, avoids comparisson.
        string = string.lower()

        # Converting the sentence into a list of words
        string_list = []
        
        for word in string.split():
            # Checks if the classifier should remove stop-words
            if self.stop_words:
                # Checks if the word is a stop-word
                if word not in self.stop_words_list:
                    # Checks if the classifier should stemmize the words
                    if self.stem:
                        string_list.append(self.stemmer.stem(word))
                    else:
                        string_list.append(word)
            else:
                # Checks if the classifier should stemmize the words
                if self.stem:
                    string_list.append(self.stemmer.stem(word))
                else:
                    string_list.append(word)
                    
        
        # Returns the n-gram list of the words
        return self._create_gram(string_list)

    # Function that creates the n-gram list of the words
    def _create_gram(self, words_list):
        bigram = []
        # The self.n_gram variable defines how many words will be linked together
        for n in range(len(words_list) + 1 - self.n_gram):
            bigram.append(" ".join(words_list[n:n+self.n_gram]))
        return bigram

    # Function that create a dictionary with the words that appear in a sentence, and its frequencies
    def _create_dict(self, sentences_series):

        
        count = {}

        for sentence in sentences_series:
            # Cleans sentence prior to counting the frequencies
            words_list = self._clean_sentence(sentence)
            for word in words_list:
                if word in count:
                    count[word] += 1
                else:
                    count[word] = 1
                        
        return count
    
    # Function that calculates the d, variable used on the Smoothing of the Probability
    def _get_d(self, df, x_label):
        
        words = []
        for sentence in df[x_label]:
            for word in self._clean_sentence(sentence):
                if word not in words:
                    words.append(word)
                    
        # It returns the total words of the dataset
        return len(words)
    
    # Function that calculates the probability of a specific category
    def _calc_prob(self, sentence, e):
        
        """
        We are using log for the probabilities to get a higher accuracy on the calculation
        If you don't use log:
            P(sentence|category) = P(word1|category)*P(word2|category)*...*P(lastword|category)
        Applying log:
            log(P(sentence|category)) = log(P(word1|category)) + log(P(word2|category)) + ... + log(P(lastword|category))
        """
        
        # Starts the probability with the probability of the specific category
        prob = log(self.classes_dicts[e]["class_prob"])
    
        # Alpha factor for the LaPlace smoothing
        total = self.classes_dicts[e]["n_words"] + self.alpha*self.d
        
        # Calculates the probability for each word (or n-gram) in the cleaned sentence
        for word in self._clean_sentence(sentence):
            if word in self.classes_dicts[e]["words"]:
                count = self.classes_dicts[e]["words"][word] + self.alpha
            else:
                count = self.alpha
            prob += log(count/total)
        
        return prob
    
    # Function that classifies the sentence
    def _classify(self, sentence):
        
        # Variable that stores the highest probability and which category it represents.
        highest = [None, None]
        
        # Calculates the probability for each category
        for e in self.classes:
            classes_probs = self._calc_prob(sentence, e)
            if highest[0] is not None:
                # Checks if the probability for that category is higher than the highest probability 
                if classes_probs > highest[1]:
                    highest[0] = e
                    highest[1] = classes_probs
            # It runs on the first iteration of the for loop. To initialize the 'highest' list
            else:
                highest[0] = e
                highest[1] = classes_probs
                
        # Returns the classification for that sentence
        return highest[0]            
    
    # This function is used to "teach" the classifier based on the training data
    def fit(self, df, x_label, y_label):
        
        self.df = df
        self.x_label = x_label
        self.y_label = y_label
        
        # Stores the possible categories to classify
        self.classes = []
        for e in df[y_label]:
            if e not in self.classes:
                self.classes.append(e)
                
        """
        Creates a dictionary with informations of each category
        keys: values
            "words": dictionary with the informations of words (or n-grams) in the category
            "n-words": number of words (or n-grams) in the category
            "class-prob": probability of that specific category
        """
        self.classes_dicts = {}
        
        # Completes classes_dicts
        for e in self.classes:
            self.classes_dicts[e] = {}
            self.classes_dicts[e]["words"] = self._create_dict(df[df[y_label] == e][x_label])
            self.classes_dicts[e]["n_words"] = len(self.classes_dicts[e]["words"])
            if self.class_prob == None:
                self.classes_dicts[e]["class_prob"] = df[df[y_label] == e][x_label].count()/df[x_label].count()
            elif self.class_prob == "equal":
                self.classes_dicts[e]["class_prob"] = 1/len(self.classes)
            
        # Creates the d (for the smoothing)
        self.d = self._get_d(df, x_label)
            
    # Function used to predict the classification of a specific series
    def predict(self, sentence_series):
        
        # List with the predictions
        predictions = []
        
        # Classify each sentence
        for sentence in sentence_series:
            predictions.append(self._classify(sentence))
            
        # Returns the classifications as a series -> easier to manipulate later with the df
        return pd.Series(predictions)
    
    # Evaluates the classifier performance
    def evaluate(self, y_test, y_pred):
        
        # Count for the correct predictions
        count = 0
        
        # Compares the predictions with the real classifications
        for e in range(len(y_test)):
            if y_test.loc[e] == y_pred.loc[e]:
                count += 1
                
        # Returns a tuple with the Accuracy and the number of correct predictions
        performance = count/(y_test.count())
        return (performance, count)
    
    """
    Creates a confusion_matrix for the classifier predictions
    Problem to be resolved -> does not work with categories that are not numbered and started on 0
    """
    def confusion_matrix(self, y_test, y_pred):
        
        n = [[0] * len(self.classes)] * len(self.classes)
        cm = np.array(n)
        
        for e in range(len(y_test)):
            cls = y_test.loc[e]
            pred = y_pred.loc[e]
            cm[cls][pred] += 1
            
        return cm

In [4]:
# Creating the classifier
classifier = NaiveBayesClassificator(n_gram=2, alpha=1, class_prob=None, stop_words=False)

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


<h4>Por que não usar o classificador para obter mais amostras de treinamento?</h4>

<p>Classificando manualmente as amostras de treinamento, criamos um referencial para nosso classificador, “ensinando-o” o que considerar irrelevante, bom ou ruim. Em outras palavras criamos um data frame em que o classificador possa se embasar ao classificar novas amostras. </p>

<p>Uma vez que as amostras de treinamento têm essa utilidade, seria imprudente utilizar o próprio classificador para gerar mais amostras para ele próprio treinar. O usuário não teria como saber se a classificação feita pelo programa é correta, pois o programa por si só não é capaz de distinguir as classes positivas, negativas e irrelevantes  sem embasamento em amostras classificadas pelo o usuário</p>

<h4>Diferentes cenários onde pode ser ultilizado Naïve Bayes</h4>

* Desenvolvimento de um programa para detecção de falhas de um manipulador robótico:

   <p> Alguns manipuladores robóticos podem apresentar problemas de calibração e em muitos casos são desenvolvidos programas para identificar essas falhas. O classificador de Naive Bayes pode ser útil para distinguir a precisão desse programa e se realmente identificou um erro "relevante". Uma demonstração formal dessa ultilização pode ser encontrada no link abaixo </p>

    <p>http://abcm.org.br/upload/files/PII_I_03%281%29.pdf</p>

    <p>É possível observar nesse ensaio que foi classficado cada tipo de erro que o programa detecta. Assim foram feitos testes para cada classe de erros e ,a partir da observação dos testes, calcular a precisão do programa. Com isso, é possível usar a lógica do classificador de Naive Bayes para ter a probabilidade da  identificação de erros que o programa detecta (neste caso, o programa tem 83% de acerto na identificação de erros).</p> 
    


